## Домашнее задание 6

Дедлайн: 1 июня 2020 23:59

В этом задании требуется реализовать инструмент для нахождения кратчайшего пути от одной статьи в русской Википедии до другой. 

За это задание можно получить максимум 13 баллов.

## Часть 1 (1 балл)

Ниже приведена функция, принимающая на вход HTML-код страницы Википедии и отдающая все ссылки на другие статьи из русской Википедии, содержащиеся в этом коде.

У этой функции есть как минимум два недостатка:

* Если мы возьмем результат работы этой функции и отберем только уникальные ссылки, у нас может произойти ситуация, когда в нашем множестве ссылок есть как ссылка вида `/wiki/какая-то_статья`, так и `/wiki/какая-то_статья#какой-то_подраздел` (хотя на самом деле это одна и та же статья).
* Эта функция отдает ссылки не только на статьи, но и на служебные страницы (`/wiki/Служебная:Свежие_правки`) и, например, файлы (`/wiki/Файл:Calabi_yau_formatted.svg`). Эти ссылки нам не нужны.

Доработайте функцию, исправив перечисленные недостатки.

In [0]:
from bs4 import BeautifulSoup # pip install bs4
from typing import List
import requests # pip install requests
import urllib

In [0]:
def links_from_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    content = soup.find("div", {"id": "mw-content-text"})

    links = content.find_all("a")

    for link in links:
        href = link.get('href', '')

        if href.startswith('/wiki'):
            if ':' not in href:

                main_link = 'https://ru.wikipedia.org' + href.split("#")[0]

                yield urllib.parse.unquote(main_link)

In [3]:
start_url = 'https://ru.wikipedia.org/wiki/Теория струн'

for url in links_from_text(requests.get(start_url).text):
    print(url)

https://ru.wikipedia.org/wiki/Теория_суперструн
https://ru.wikipedia.org/wiki/Теория_суперструн
https://ru.wikipedia.org/wiki/Теория_бозонных_струн
https://ru.wikipedia.org/wiki/М-теория
https://ru.wikipedia.org/wiki/Гетеротическая_струна
https://ru.wikipedia.org/wiki/Голографический_принцип
https://ru.wikipedia.org/wiki/Квантовая_струна
https://ru.wikipedia.org/wiki/Брана
https://ru.wikipedia.org/wiki/Пространство_Калаби_—_Яу
https://ru.wikipedia.org/wiki/D-брана
https://ru.wikipedia.org/wiki/E₈_(математика)
https://ru.wikipedia.org/wiki/Суперсимметрия
https://ru.wikipedia.org/wiki/Супергравитация
https://ru.wikipedia.org/wiki/Квантовая_гравитация
https://ru.wikipedia.org/wiki/Виттен,_Эдвард
https://ru.wikipedia.org/wiki/Венециано,_Габриеле
https://ru.wikipedia.org/wiki/Грин,_Брайан_Рэндолф
https://ru.wikipedia.org/wiki/Гросс,_Дейвид
https://ru.wikipedia.org/wiki/Каку,_Митио
https://ru.wikipedia.org/wiki/Малдасена,_Хуан
https://ru.wikipedia.org/wiki/Поляков,_Александр_Маркович
https:/

## Часть 2 (5 баллов)

Реализуйте заданную функцию.

Под путем подразумевается последовательность ссылок на статьи из **ru.wikipedia.org** (то есть нам не подходит случай, когда страница A из ru.wikipedia.org ссылается на страницу B из en.wikipedia.org, а та, в свою очередь, на страницу C из ru.wikipedia.org).

Проще всего реализовать функцию через поиск в ширину (https://en.wikipedia.org/wiki/Breadth-first_search):

1. Начинаем из исходной статьи
2. Отбираем все статьи, куда она ссылается
3. Если есть ссылка на конечную статью - завершаем
4. Если нет ссылки на конечную статью - отбираем все статьи, куда они ссылаются **и где мы еще не были**
5. Если есть ссылка на конечную статью - завершаем
6. Если нет - повторяем до победного

Чтобы вернуть сам путь, а не только его длину, вам нужно хранить отношения между статьями. Придумайте структуру, которая позволит быстро восстанавливать путь.

Запросы к серверу должны выполняться параллельно (максимальное количество одновременно бегущих запросов задаётся параметром `n_threads`). Вы можете использовать любой из доступных инструментов (например, практически везде есть некоторый объект Pool с функцией map):

* threading
* multiprocessing
* asyncio
* gevent


При выборе инструмента нужно подумать (https://habr.com/ru/post/421625/). Асинхронные библиотеки и threading показывают высокую производительность при выполнении HTTP-запросов (поскольку эта задача нагружает не процессор, а устройства ввода-вывода, то есть это I/O bound задача). Но после выполнения запросов нужно извлечь ссылки из текстов страниц, а это, вообще-то, достаточно вычислительно сложная задача (все библиотеки для парсинга HTML, по сути, превращают строку с HTML-кодом во внутреннее представление дерева элементов страницы).

Поэтому если вы будете параллелить еще и парсинг HTML (а это происходит внутри функции `links_from_text`, когда мы создаем класс BeautifulSoup), это нужно делать либо с помощью multiprocessing (чтобы задача параллелилась на разные ядра процессора), либо не параллелить вообще и просто доставать ссылки последовательно в один поток.


In [0]:
from concurrent.futures import ProcessPoolExecutor, as_completed

In [5]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(process)d - %(asctime)s - %(message)s',
)
logging.info('test')

1379 - 2020-05-31 21:25:21,941 - test


In [0]:
def get_text(path):
    # logging.info('getting text %s', path[-1])
    url = path[-1]
    return path, requests.get(url).text

In [0]:
def get_links(path, text, end_url):
    # logging.info('getting children %s', path[-1])
    children = []
    links = links_from_text(text)
    for link in links:
        if link == end_url:
            # logging.info('found in %s', url)
            return path, None
        else:
            children.append(link)
    out = [path + [child] for child in set(children)]
    return path, out

In [0]:
def shortest_path(from_article: str, to_article: str, n_threads: int = 10) -> List[str]:
    if from_article == to_article:
        return []

    start_url = 'https://ru.wikipedia.org/wiki/' + from_article
    end_url = 'https://ru.wikipedia.org/wiki/' + to_article

    for url in [start_url, end_url]:
        code = requests.get(url).status_code
        if code == 404:
            raise ValueError(f'{url} does not exist')
        elif 299 < code < 200:
            raise SystemError(f'could not open {url}, got code %d', code)

    executor_text = ProcessPoolExecutor(max_workers=n_threads)
    executor_links = ProcessPoolExecutor(max_workers=n_threads)

    queue = [[start_url]]
    visited = []
    finished = []

    while queue:

        level = []

        future_texts = []
        future_links = []
        for path in queue:
            queue.remove(path)
            node = path[-1]

            if node not in visited:
                visited.append(node)
                # logging.info('executor_text submit %s', node)
                future_text = executor_text.submit(get_text, path)
                future_texts.append(future_text)

        for f_text in as_completed(future_texts):
            path, text = f_text.result()
            # logging.info('executor_link submit %s', path[-1])
            future_link = executor_links.submit(get_links, path, text, end_url)
            future_links.append(future_link)

        for f_links in as_completed(future_links):

            res = f_links.result()

            parent = res[0]
            children = res[1]
            # logging.info('checking %s', parent[-1])

            finished.append(parent[-1])

            if children is None:
                logging.info('finished_len=%s', len(finished))
                full_path = parent[1:] + [end_url]
                final_path = [url.replace('https://ru.wikipedia.org/wiki/', '') for url in full_path]
                logging.info(f'path={final_path}')
                return final_path
            else:
                level.extend(children)
        queue.extend(level)

In [12]:
%%time
assert len(shortest_path('Теория_струн', 'Теория_струн', 10)) == 0

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


In [16]:
%%time
assert shortest_path('Теория_струн', 'Теоретическая_физика') == ['Теоретическая_физика'] # не включайте исходную статью в путь

1379 - 2020-05-31 21:30:31,464 - finished_len=1
1379 - 2020-05-31 21:30:31,466 - path=['Теоретическая_физика']


CPU times: user 59.7 ms, sys: 104 ms, total: 164 ms
Wall time: 838 ms


In [15]:
%%time
assert len(shortest_path('Теория_струн', 'Динамический_хаос', 10)) > 0

1379 - 2020-05-31 21:30:12,676 - finished_len=103
1379 - 2020-05-31 21:30:12,679 - path=['Гравитация', 'Динамический_хаос']


CPU times: user 346 ms, sys: 236 ms, total: 582 ms
Wall time: 9.73 s


## Часть 3 (2 балла)

### Немного про аргументы командой строки

Они позволяют задавать параметры для выполнения программы прямо в терминале. Например, когда вы выполняете команду `git add file.txt`, вы на самом деле вызываете команду `git`, передавая ей **позиционные** аргументы (поскольку порядок их перечисления важен для работы программы) `add` и `file.txt`. **Непозиционные** аргументы передаются, например, так:

`python3 my_script.py --first_arg=123 --second_arg=1111`

причем мы можем поменять их местами при вызове команды и в логике работы программы ничего не изменится:

`python3 my_script.py --second_arg=1111 --first_arg=123`


### Задание

1. Закройте Jupyter и откройте PyCharm.
2. Используйте библиотеку `click` (https://click.palletsprojects.com/en/7.x/), чтобы написать .py-скрипт, принимающий исходную и конечную статью как аргументы командной строки. Пример того, как программа должна вызываться:

`python3 wikidistance.py --to=Теоретическая_физика --from=Теория_струн` (если вы используете непозиционные аргументы)

`python3 wikidistance.py Теория_струн Теоретическая_физика` (если вы используете позиционные аргументы, вы должны перечислять их в определенном порядке)

## Часть 4 (2 балла)

Доработайте функцию и программу, чтобы она выводила **все пути** минимальной длины.

## Часть 5 (бонусная; +3 балла к итогу)

NB: Если вы выполняете эту часть, то все равно сдайте отдельно программу, полученную на этапе 3 или 4.

Доработайте функцию и программу, чтобы программа принимала на вход путь до JSON-файла с заданием. Пример вызова:

`python3 wikidistance_pro_edition.py ~/path/to/task.json ~/path/to/output.json`

Здесь в файле, который находится по пути `ваша_домашняя_папка/path/to/task.json`, лежит JSON-файл следующего вида:

```json
[
{"from": "Теория_струн", "to": "Теоретическая_физика"},
{"from": "Теория_всего_(философия)", "to": "Теория_струн"}
]
```

Программа должна обходить Википедию до тех пор, пока не найдет все кратчайшие маршруты между всеми переданными парами статей. Она должна использовать общую структуру данных для всех пар (чтобы в итоге сделать меньше запросов к серверу, чем если бы мы находили путь для каждой из пар по-отдельности).

Программа должна положить в выходной файл, путь к которому задается параметром командой строки (можно использовать file arguments из click https://click.palletsprojects.com/en/7.x/arguments/), JSON вида:

```json
[
{"from": "Теория_струн", "to": "Теоретическая_физика", "all_shortest_paths": [["a", "b", "Теоретическая_физика"]]},
{"from": "Теория_всего_(философия)", "to": "Теория_струн", "all_shortest_paths": [["с", "b", "Теория_струн"], ["d", "b", "Теория_струн"]]}
]
```

Здесь в `all_shortest_paths` лежит список списков строк - все возможные кратчайшие пути.


Основной челлендж этой части - аккуратно реализовать работу с представлением графа Википедии.

## Дополнительные советы и требования

* PEP8 ¯¯\\\_(ツ)\_/¯¯
* Предполагается, что исходная и конечная статья существуют (не требуется это явно проверять, но если очень хочется, то можно).
* Если при загрузке какой-то из страниц возникает ошибка, нужно повторить запрос. Если это происходит слишком часто, возможно, вы DDoS-ите сайт запросами и стоит уменьшить количество потоков.
* Чтобы объединить несколько итераторов, можно использовать функцию https://docs.python.org/3.8/library/itertools.html#itertools.chain (и вообще в этом модуле есть много всего полезного)
* При тестировании не используйте слишком далёкие по смыслу статьи - есть вероятность, что вы будете очень долго ждать выполнения программы (http://www.machinelearning.ru/wiki/index.php?title=%D0%9F%D1%80%D0%BE%D0%BA%D0%BB%D1%8F%D1%82%D0%B8%D0%B5_%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%80%D0%BD%D0%BE%D1%81%D1%82%D0%B8)